In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hotel-reviews/nasket_data_by_sentences-2020-04-15 (1).xlsx


### Read data

In [2]:
df = pd.read_excel('/kaggle/input/hotel-reviews/nasket_data_by_sentences-2020-04-15 (1).xlsx')
df.head(10)

,id,sentence,Overall Sentiment\n(Positive/Negative/Neutral),"Services & Staff\n(0 = not mentioned, \n1 = mentioned)",Services & Staff Sentiment,"Amenities Sentiment(0 = not mentioned, \n1 = mentioned)",Amenities Sentiment,"Hotel Condition \n(0 = not mentioned, \n1 = mentioned)",Hotel Condition Sentiment,"Cleanliness\n(0 = not mentioned, \n1 = mentioned)",Cleanliness Sentiment
0,34437648,5 Stories with no elevator,Negative,1.0,Negative,0.0,Negative,1.0,Negative,0.0,Negative
1,34437648,Great location right at Nana BTS Station!,Positive,1.0,Positive,0.0,Positive,1.0,Positive,0.0,Positive
2,34437648,Room was modern and clean eventhough small.,Positive,1.0,Positive,0.0,Positive,1.0,Positive,1.0,Positive
3,34437648,Only gripe... stairs only and my room was on t...,Negative,0.0,NaN,0.0,NaN,1.0,Negative,0.0,Negative
4,34437886,Dump,Negative,0.0,NaN,0.0,NaN,1.0,Negative,0.0,Negative
5,34437886,pretty run down old hotel.,Negative,0.0,NaN,0.0,NaN,1.0,Negative,0.0,NaN
6,34437886,its nothing like what the photos depict.,Negative,0.0,NaN,0.0,Negative,1.0,Negative,0.0,NaN
7,34437886,book elsewhere.,Negative,0.0,NaN,0.0,Negative,1.0,Negative,0.0,NaN
8,36080259,"Amazing service, amazing staff, amazing room",Positive,1.0,Positive,0.0,Positive,1.0,Positive,0.0,NaN
9,36080259,This is hotel is conveniently located next to ...,Positive,0.0,Positive,0.0,Positive,0.0,NaN,0.0,NaN


### Filter data

In [3]:
reviews_df = df[['sentence','Overall Sentiment\n(Positive/Negative/Neutral)']]
reviews_df.head(10)

,sentence,Overall Sentiment\n(Positive/Negative/Neutral)
0,5 Stories with no elevator,Negative
1,Great location right at Nana BTS Station!,Positive
2,Room was modern and clean eventhough small.,Positive
3,Only gripe... stairs only and my room was on t...,Negative
4,Dump,Negative
5,pretty run down old hotel.,Negative
6,its nothing like what the photos depict.,Negative
7,book elsewhere.,Negative
8,"Amazing service, amazing staff, amazing room",Positive
9,This is hotel is conveniently located next to ...,Positive


## Pre-processing

In [4]:
import re
reviews_df['sentence'] = reviews_df['sentence'].str.replace('[^\w\s]','')
reviews_df['sentence'] = reviews_df['sentence'].str.replace('\d+', '')
reviews_df['sentence'] = reviews_df['sentence'].str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

### Drop empty cells

In [5]:
drop = reviews_df[pd.isnull(reviews_df['Overall Sentiment\n(Positive/Negative/Neutral)'])].index
reviews_df.drop(drop , inplace=True)
reviews_df = reviews_df.reset_index(drop = True) 
reviews_df.head(10)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,sentence,Overall Sentiment\n(Positive/Negative/Neutral)
0,stories with no elevator,Negative
1,great location right at nana bts station,Positive
2,room was modern and clean eventhough small,Positive
3,only gripe stairs only and my room was on the ...,Negative
4,dump,Negative
5,pretty run down old hotel,Negative
6,its nothing like what the photos depict,Negative
7,book elsewhere,Negative
8,amazing service amazing staff amazing room,Positive
9,this is hotel is conveniently located next to ...,Positive


In [6]:
reviews_df['sentence'].replace('', np.nan, inplace=True)
drop = reviews_df[pd.isnull(reviews_df['sentence'])].index
reviews_df.drop(drop , inplace=True)
reviews_df = reviews_df.reset_index(drop = True) 
reviews_df.head(10)

,sentence,Overall Sentiment\n(Positive/Negative/Neutral)
0,stories with no elevator,Negative
1,great location right at nana bts station,Positive
2,room was modern and clean eventhough small,Positive
3,only gripe stairs only and my room was on the ...,Negative
4,dump,Negative
5,pretty run down old hotel,Negative
6,its nothing like what the photos depict,Negative
7,book elsewhere,Negative
8,amazing service amazing staff amazing room,Positive
9,this is hotel is conveniently located next to ...,Positive


### Remove stop words

In [7]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [8]:
reviews_df['no_sw'] = reviews_df['sentence'][:].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [9]:
reviews_df.head(10)

,sentence,Overall Sentiment\n(Positive/Negative/Neutral),no_sw
0,stories with no elevator,Negative,stories elevator
1,great location right at nana bts station,Positive,great location right nana bts station
2,room was modern and clean eventhough small,Positive,room modern clean eventhough small
3,only gripe stairs only and my room was on the ...,Negative,gripe stairs room top floor th floor really ho...
4,dump,Negative,dump
5,pretty run down old hotel,Negative,pretty run old hotel
6,its nothing like what the photos depict,Negative,nothing like photos depict
7,book elsewhere,Negative,book elsewhere
8,amazing service amazing staff amazing room,Positive,amazing service amazing staff amazing room
9,this is hotel is conveniently located next to ...,Positive,hotel conveniently located next sky train exit...


### Spelling correction

In [10]:
# !pip install pyspellchecker
# from spellchecker import SpellChecker

# spell = SpellChecker()

## Bag of words (nltk, manual method)

In [11]:
import nltk
from nltk import pos_tag, word_tokenize


sentence_vectors = []
wordfreq = {}
for sentence in reviews_df['no_sw']:
    tokens = word_tokenize(sentence)
    sent_vec = []
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

### Select number of most frequent words to use

In [12]:
import heapq
num_features = 2000
most_freq = heapq.nlargest(num_features, wordfreq, key=wordfreq.get)

### Vectorize

In [13]:
sentence_vectors = []
for sentence in reviews_df['no_sw']:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [14]:
sentence_vectors = np.asarray(sentence_vectors)
sentence_vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

## Building Model

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

labels = reviews_df['Overall Sentiment\n(Positive/Negative/Neutral)']
labels = np.array(list(map(lambda x: 1 if x=="Positive" else 0, labels)))
X_train, X_test, y_train, y_test = train_test_split(sentence_vectors, labels, test_size=0.2, random_state=0)

### Sample with RFC

In [16]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [17]:
predictions = text_classifier.predict(X_test)

### Evaluations

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[449 177]
 [154 306]]
              precision    recall  f1-score   support

           0       0.74      0.72      0.73       626
           1       0.63      0.67      0.65       460

    accuracy                           0.70      1086
   macro avg       0.69      0.69      0.69      1086
weighted avg       0.70      0.70      0.70      1086

0.6952117863720073


## Deep Learning

In [19]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

In [20]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                20010     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 20,021
Trainable params: 20,021
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(X_train, y_train,
                    epochs=25,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/25
434/434 [==============================] - 1s 3ms/step - loss: 0.6685 - accuracy: 0.6401 - val_loss: 0.6269 - val_accuracy: 0.7017
Epoch 2/25
434/434 [==============================] - 1s 2ms/step - loss: 0.5756 - accuracy: 0.7403 - val_loss: 0.5682 - val_accuracy: 0.7348
Epoch 3/25
434/434 [==============================] - 1s 2ms/step - loss: 0.5024 - accuracy: 0.7806 - val_loss: 0.5522 - val_accuracy: 0.7468
Epoch 4/25
434/434 [==============================] - 1s 2ms/step - loss: 0.4544 - accuracy: 0.8046 - val_loss: 0.5542 - val_accuracy: 0.7477
Epoch 5/25
434/434 [==============================] - 1s 2ms/step - loss: 0.4189 - accuracy: 0.8196 - val_loss: 0.5621 - val_accuracy: 0.7376
Epoch 6/25
434/434 [==============================] - 1s 2ms/step - loss: 0.3903 - accuracy: 0.8376 - val_loss: 0.5734 - val_accuracy: 0.7302
Epoch 7/25
434/434 [==============================] - 1s 2ms/step - loss: 0.3663 - accuracy: 0.8447 - val_loss: 0.5894 - val_accuracy: 0.7228
Epoch 

In [23]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9295
Testing Accuracy:  0.7182
